In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [6]:
data_fake = pd.read_csv('FakeNews.csv')
data_true = pd.read_csv('TrueNews.csv')

In [7]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [22]:
data_fake["class"] = 0
data_true["class"] = 1

In [23]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [25]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i], axis = 0, inplace = True)
    
data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i], axis = 0, inplace = True)

In [26]:
data_fake.shape, data_true.shape

((23461, 5), (21397, 5))

In [27]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

C:\Users\Parth Magdum\AppData\Local\Temp\ipykernel_20472\277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
C:\Users\Parth Magdum\AppData\Local\Temp\ipykernel_20472\277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [28]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [29]:
 data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [30]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [31]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [32]:
data = data.sample(frac = 1)

In [33]:
data.head()

,text,class
20384,(Reuters) - Several prominent Saudi clerics ha...,1
14072,NAYPYITAW (Reuters) - Myanmar and Bangladesh s...,1
8046,WASHINGTON (Reuters) - The White House on Frid...,1
4346,WASHINGTON (Reuters) - Advocates of landmark n...,1
21144,MEXICO CITY (Reuters) - Trade negotiators from...,1


In [34]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [35]:
data.columns

Index(['text', 'class'], dtype='object')

In [36]:
data.head()

,text,class
0,(Reuters) - Several prominent Saudi clerics ha...,1
1,NAYPYITAW (Reuters) - Myanmar and Bangladesh s...,1
2,WASHINGTON (Reuters) - The White House on Frid...,1
3,WASHINGTON (Reuters) - Advocates of landmark n...,1
4,MEXICO CITY (Reuters) - Trade negotiators from...,1


In [37]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [38]:
data.columns

Index(['text', 'class'], dtype='object')

In [39]:
data.head()

,text,class
0,(Reuters) - Several prominent Saudi clerics ha...,1
1,NAYPYITAW (Reuters) - Myanmar and Bangladesh s...,1
2,WASHINGTON (Reuters) - The White House on Frid...,1
3,WASHINGTON (Reuters) - Advocates of landmark n...,1
4,MEXICO CITY (Reuters) - Trade negotiators from...,1


In [40]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', " ", text)
    text = re.sub('https?://\S+|www.\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text;

In [41]:
data['text'] = data['text'].apply(wordopt)

In [42]:
x = data['text']
y = data['class']

In [43]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [47]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [48]:
pred_lr = LR.predict(xv_test)

In [49]:
LR.score(xv_test, y_test)

0.9858225590726706

In [51]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5849
           1       0.98      0.99      0.99      5366

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



In [52]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [53]:
pred_dt = DT.predict(xv_test)

In [54]:
DT.score(xv_test, y_test)

0.995720017833259

In [55]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5849
           1       1.00      0.99      1.00      5366

    accuracy                           1.00     11215
   macro avg       1.00      1.00      1.00     11215
weighted avg       1.00      1.00      1.00     11215



In [57]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [58]:
pred_gb = GB.predict(xv_test)

In [59]:
GB.score(xv_test, y_test)

0.9945608559964334

In [60]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5849
           1       0.99      1.00      0.99      5366

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



In [61]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [62]:
pred_rf = RF.predict(xv_test)

In [79]:
RF.score(xv_test, y_test)

0.9881408827463218

In [64]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5849
           1       0.99      0.99      0.99      5366

    accuracy                           0.99     11215
   macro avg       0.99      0.99      0.99     11215
weighted avg       0.99      0.99      0.99     11215



In [72]:
def output_lable(n):
    if n == 0 :
        return "This news is Fake"
    elif n == 1:
        return "This news is not Fake"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_x_test = new_def_test['text']
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)
    
    return print("\n\nLR Prediction : {} \nDT Prediction : {} \nGBC Prediction : {} \nRF Prediction : {}".format(output_lable(pred_LR[0]),
                                                                                                                output_lable(pred_DT[0]),
                                                                                                                output_lable(pred_GB[0]),
                                                                                                                output_lable(pred_RF[0])))

In [ ]:
#For Testing of Models

news = str(input())
manual_testing(news)